[한국전력공사\_공동주택 전력·기상 융합데이터\_20221231 | 공공데이터포털](https://www.data.go.kr/data/15120804/fileData.do)

한국전력공사_공동주택 전력·기상 융합데이터
한국전력공사와 기상청이 공동주택 부하관리관련 연구개발 활성화를 목적으로 제공하는 데이터셋입니다.
데이터셋은 기상청 동네예보 격자단위 시간대별 공동주택 전력부하와 기상관측값을 포함하고 있습니다.
공동주택 전력부하는 각 연도별 결측치가 없는 단지에 한하여 합계한 통계값이며, 격자내 공동주택이 10개 이상인 격자에 대해서만 제공하고 있습니다.

[2024 날씨 빅데이터콘테스트 설명동영상(과제4, 전력분야) - YouTube](https://www.youtube.com/watch?v=9xCBKNrymig&t=265s)


---

**| 학습 데이터(전력)**

-   **제공기간** : 2020년, 2021년, 2022년 (총 3개년)
-   **자료설명** : 원격검침이 이루어지는 공동주택 중 각 연도별 결측치가 없는 단지를 대상으로 기상예보 격자별로 산출된 전력 통계값. 격자내 공동주택이 10개 이상인 격자에 대해서만 공개함.

**컬럼 정보**

| 변수명        | 변수 설명      | 항목설명                                                                                                |
| :------------ | :------------- | :------------------------------------------------------------------------------------------------------ |
| TM            | 날짜           | 공동주택 전력부하 측정 날짜(시간포함, 단위(0~23시))                                                          |
| HH24          | 시간           | 공동주택 전력부하 측정 시간(1~24), 5시는 4시 1분~5시 00분까지의 전력부하를 의미                                  |
| weekday       | 요일           | 요일을 숫자형식으로 표시 월요일(0)~일요일(6)                                                                 |
| week\_name    | 주중 주말      | 주중 주말을 숫자형식으로 표시 주중(0), 주말(1)                                                               |
| SUM\_QCTR     | 계약전력합계   | 해당격자의 전력통계 산출에 포함된 공동주택의 계약전력 합계                                                       |
| N             | 공동주택 수    | 해당격자의 전력통계 산출에 포함된 공동주택의 수, 단위(단지)                                                      |
| SUM\_LOAD     | 전력수요 합계  | 해당격자/시각에 측정된 공동주택의 전력수요 합계                                                                |
| N\_MEAN\_LOAD | 전력부하량 평균 | 격자내 총 전력부하량을 아파트 수로 나누어 격자의 평균 부하량을 산출                                                 |
| elec          | 전력기상지수   | 해당격자의 공동주택의 연평균 부하량을 100으로 했을 때, 해당 시각에 예상되는 부하량을 상대적인 수치로 표현                     |

---

In [ ]:
print("--- 라이브러리 임포트 및 환경 설정 ---")
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib  # 한글 폰트 깨짐 방지
import warnings

# 모델링 및 평가 관련 라이브러리
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

## 데이터 로드

In [ ]:
# --- 데이터 로딩 ---
print("\n--- 데이터 로딩 ---")
DATA_FOLDER = 'data'
csv_files = [f for f in os.listdir(DATA_FOLDER) if f.endswith('.csv')]
if not csv_files:
    raise FileNotFoundError(f"'{DATA_FOLDER}' 폴더에 분석할 CSV 파일이 없습니다.")

df_list = []
for csv_file in csv_files:
    file_path = os.path.join(DATA_FOLDER, csv_file)
    try:
        df_temp = pd.read_csv(file_path, encoding="cp949")
        df_list.append(df_temp)
    except Exception as e:
        print(f"'{csv_file}' 파일 로딩 실패: {e}")

df_raw = pd.concat(df_list, ignore_index=True).sample(frac=0.01).reset_index(drop=True)
df_raw.to_csv("data/kepco_ml.csv", index=False)
print(f"데이터 로딩 완료. 총 {len(df_raw)}개 행.")

In [ ]:
df_raw

In [ ]:
df_raw.info()

## 데이터 전처리

In [ ]:
print("\n--- 데이터 전처리 ---")
# '시' 컬럼이 24시인 경우, 0시로 변경
df_raw['시'] = df_raw['시'].replace(24, 0)

# datetime 객체 생성 및 인덱스 설정
# 컬럼명이 '연도', '월', '일', '시'이므로, to_datetime에 format을 명시적으로 지정
df_raw['datetime'] = pd.to_datetime(
    df_raw['연도'].astype(str) + '-' +
    df_raw['월'].astype(str).str.zfill(2) + '-' +
    df_raw['일'].astype(str).str.zfill(2) + ' ' +
    df_raw['시'].astype(str).str.zfill(2) + ':00:00',
    format='%Y-%m-%d %H:%M:%S'
)
df_processed = df_raw.sort_values('datetime').set_index('datetime')

# 결측치 확인 및 처리 (시계열 데이터이므로 ffill 사용)
print("결측치 처리 전:\n", df_processed.isnull().sum().loc[lambda x: x > 0])
df_processed.fillna(method='ffill', inplace=True)
print("\n결측치 처리 완료.")

## 탐색적 데이터 분석 (EDA)

In [ ]:
# 분석에 사용할 주요 변수 설정
TARGET_COLUMN = '전력부하합계'
split_index = int(len(df_processed) * 0.8)
SPLIT_DATE = df_processed.index[split_index]
print(f"데이터 분할 인덱스: {split_index}")

In [ ]:
df_processed.describe()

In [ ]:
df_processed.hist(bins=50, figsize=(20, 15));

In [ ]:
# 시각화를 통해 데이터 패턴 파악 (분석 대상 연도 기준)
df_eda = df_processed.copy()
fig, axes = plt.subplots(1, 2, figsize=(18, 6))
# 월별 평균 전력부하
df_eda.groupby(
    df_eda.index.month
    )['전력부하합계'].mean().plot(ax=axes[0], marker='o', title='월별 평균 전력부하합계')
axes[0].set_xlabel('월')
axes[0].set_ylabel('평균 전력부하합계')
axes[0].grid(True)
# 시간대별 평균 전력부하
df_eda.groupby(
    df_eda.index.hour
    )['전력부하합계'].mean().plot(ax=axes[1], marker='o', title='시간대별 평균 전력부하합계')
axes[1].set_xlabel('시')
axes[1].set_ylabel('평균 전력부하합계')
axes[1].grid(True)
plt.tight_layout()
plt.show()
print("시간에 따른 전력부하 패턴 시각화")

## 특성 공학

In [ ]:
# --- 특성 공학 (Feature Engineering) ---
print("\n--- 특성 공학 (Feature Engineering) ---")

# df_processed에 직접 특성 추가
df_processed['월'] = df_processed.index.month
df_processed['시'] = df_processed.index.hour
df_processed['요일'] = df_processed.index.dayofweek

# 주기성 특성 (Sin/Cos 변환)
df_processed['시간_sin'] = np.sin(2 * np.pi * df_processed['시'] / 24)
df_processed['시간_cos'] = np.cos(2 * np.pi * df_processed['시'] / 24)
df_processed['월_sin'] = np.sin(2 * np.pi * df_processed['월'] / 12)
df_processed['월_cos'] = np.cos(2 * np.pi * df_processed['월'] / 12)

# 파생 변수
df_processed['주말'] = df_processed['요일'].apply(lambda x: 1 if x >= 5 else 0)
df_processed['냉방도일'] = np.maximum(0, df_processed['기온'] - 24)
df_processed['난방도일'] = np.maximum(0, df_processed['기온'] - 18)
df_processed['불쾌지수'] = 1.8 * df_processed['기온'] - 0.55 * (1 - df_processed['상대습도']/100) * (1.8 * df_processed['기온'] - 26) + 32

# 시계열 특성 (Lag, Rolling)
for lag in [24, 168]: # 1일, 1주일 전 데이터
    df_processed[f'전력부하_lag_{lag}h'] = df_processed[TARGET_COLUMN].shift(lag)
    df_processed[f'기온_lag_{lag}h'] = df_processed['기온'].shift(lag)
df_processed['기온_rolling_24h'] = df_processed['기온'].rolling(window=24).mean()

# 특성 생성으로 인한 결측치는 이전 값으로 채움
df_processed.fillna(method='bfill', inplace=True)
df_featured = df_processed.copy()
print("특성 공학 완료. 최종 특성 개수:", len(df_featured.columns))


## 모델링 데이터 준비

In [ ]:
print("\n--- 모델링 데이터 준비 ---")
# 시계열 데이터를 훈련/테스트 세트로 분할
train = df_featured[df_featured.index < SPLIT_DATE]
test = df_featured[df_featured.index >= SPLIT_DATE]
print(f"훈련 데이터: {train.index.min()} ~ {train.index.max()} ({len(train)}개 행)")
print(f"테스트 데이터: {test.index.min()} ~ {test.index.max()} ({len(test)}개 행)")

# 특성(X)과 타겟(y) 분리
features = [col for col in df_featured.columns if col not in [
    TARGET_COLUMN, '연도', '월', '일', '시', '요일']]
X_train = train[features]
y_train = train[TARGET_COLUMN]
X_test = test[features]
y_test = test[TARGET_COLUMN]

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("데이터 분할 및 스케일링 완료.")

## 모델 훈련

In [ ]:
print("\n--- 모델 훈련 및 평가 ---")
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

results_list = []
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    
    results_list.append({'Model': name, 'R²': r2, 'RMSE': rmse, 'MAPE(%)': mape})
    
results_df = pd.DataFrame(results_list).set_index('Model')
print("모델별 성능 평가 결과:")
print(results_df.sort_values(by='R²', ascending=False))

## 최종 모델 분석 및 결과 시각화 

In [ ]:
print("\n--- 최종 모델 분석 및 결과 시각화 ---")
# 가장 성능이 좋았던 Random Forest를 최종 모델로 선정
final_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
final_model.fit(X_train_scaled, y_train)

# 특성 중요도 분석
importances = pd.Series(final_model.feature_importances_, index=features)
top_20_importances = importances.sort_values(ascending=False).head(20)

# 특성 중요도 시각화 
plt.figure(figsize=(10, 7))
sns.barplot(
    y=top_20_importances.index,
    x=top_20_importances.values,
    orient='h',
    palette='viridis'
)
plt.title('상위 20개 특성 중요도 (Random Forest)')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

## 모델 예측

In [ ]:
# 예측 결과 시각화 
y_pred_final = final_model.predict(X_test_scaled)
predictions = pd.DataFrame({'실제값': y_test, '예측값': y_pred_final}, index=y_test.index)

# 전체 테스트 기간 시각화 
plt.figure(figsize=(14, 5))
sns.lineplot(data=predictions[['실제값', '예측값']])
plt.title('테스트 기간 전체 예측 결과 (실제값 vs 예측값)')
plt.xlabel('날짜')
plt.ylabel('전력부하합계')
plt.tight_layout()
plt.show()

# 특정 1주일 확대 시각화 
sample_predictions = predictions.loc['2022-11-07':'2022-11-13']
plt.figure(figsize=(14, 5))
sns.lineplot(data=sample_predictions[['실제값', '예측값']], marker='o')
plt.title('테스트 기간 중 1주일 확대 (2022-11-07 ~ 2022-11-13)')
plt.xlabel('날짜 및 시간')
plt.ylabel('전력부하합계')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 결과 분석

In [ ]:
results_df

In [ ]:
# --- 결론 및 평가지표 시각화 ---

# 실제 results_df의 컬럼명을 확인하여 metrics와 metric_names를 동적으로 생성
available_metrics = [col for col in results_df.columns if col not in ['Model', 'model', 'name']]
# 대표적인 평가지표 이름 매핑 (필요시 확장)
metric_name_map = {
    'R2': 'R²',
    'R²': 'R²',
    'MAE': 'MAE',
    'MSE': 'MSE',
    'RMSE': 'RMSE',
    'MAPE': 'MAPE',
}
# 실제로 존재하는 컬럼만 사용
metrics = []
metric_names = {}
for m in ['R²', 'R2', 'MAE', 'MSE', 'RMSE', 'MAPE', 'MedAE', 'MedianAE']:
    if m in results_df.columns:
        metrics.append(m)
        metric_names[m] = metric_name_map.get(m, m)
    elif m in available_metrics:
        metrics.append(m)
        metric_names[m] = metric_name_map.get(m, m)

if not metrics:
    raise ValueError("results_df에 평가지표 컬럼이 존재하지 않습니다. 컬럼명을 확인하세요.")

print("\n--- 결론 및 모델별 평가지표 ---")
for idx, row in results_df.iterrows():
    print(f"{idx} 모델:")
    for m in metrics:
        try:
            print(f"  {metric_names[m]}: {row[m]:.4f}")
        except KeyError:
            print(f"  {metric_names[m]}: (데이터 없음)")
    print("-" * 30)

# 가장 성능이 좋은 모델 정보 (R² 또는 R2 기준)
r2_col = 'R²' if 'R²' in results_df.columns else ('R2' if 'R2' in results_df.columns else None)
if r2_col is not None:
    best_model = results_df.sort_values(by=r2_col, ascending=False).iloc[0]
    print(f"\n분석 결과, {best_model.name} 모델이 {metric_name_map[r2_col]} 점수 {best_model[r2_col]:.2f}로 가장 높은 예측 성능을 보였습니다.")
else:
    best_model = results_df.iloc[0]
    print(f"\n분석 결과, {best_model.name} 모델이 가장 높은 예측 성능을 보였습니다.")
print("주요 예측 변수로는 과거 전력부하(Lag), 계약전력합계, 기온 관련 특성 등이 확인되었습니다.")
print("모든 분석 과정이 완료되었습니다.")

# 평가지표 시각화
n_metrics = len(metrics)
nrows = 2
ncols = (n_metrics + 1) // 2
fig, axes = plt.subplots(nrows, ncols, figsize=(6 * ncols, 5 * nrows))
axes = axes.flatten()

for i, m in enumerate(metrics):
    ax = axes[i]
    bars = ax.bar(results_df.index, results_df[m], color='skyblue')
    ax.set_title(metric_names[m])
    ax.set_ylabel(metric_names[m])
    ax.set_xticklabels(results_df.index, rotation=20)
    # R²/R2는 높을수록 좋으므로 강조
    if m in ['R²', 'R2']:
        best_idx = results_df[m].idxmax()
        bars[results_df.index.get_loc(best_idx)].set_color('orange')
    else:
        best_idx = results_df[m].idxmin()
        bars[results_df.index.get_loc(best_idx)].set_color('orange')

# 빈 subplot이 있으면 숨김
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.suptitle('모델별 평가지표 비교', fontsize=18)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()